## Regular gradio setup

In [3]:
import gradio
from skimage import io
import base64
from tensorflow.keras.models import load_model
from src.moleimages import MoleImages
import tensorflow
import numpy as np
print(tensorflow.__version__)

import importlib.util


%load_ext autoreload
%autoreload 2

1.15.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
spec = importlib.util.spec_from_file_location("gradio_config_2", "second-config/gradio_config_2.py")
gradio_config_2 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gradio_config_2)

In [18]:
gradio_config_2.load()

OSError: SavedModel file does not exist at: ./models/mymodel-2.h5/{saved_model.pbtxt|saved_model.pb}

In [2]:
gradio.hosted.launch_from_config('/second-config/gradio_config_2.py')

AttributeError: module 'gradio' has no attribute 'hosted'

In [7]:
def load():
    model = load_model("./models/mymodel-2.h5")
    return model 

def predict(inp, model):
    inp = inp.split(';')[1]
    inp = inp.split(',')[1]
    if isinstance(inp, bytes):
        inp = inp.decode("utf-8")

    inp = base64.b64decode(inp)
    img = io.imread(inp, plugin='imageio')
    
    mimg = MoleImages()
    X = mimg.load_image(img)
    y_pred = model.predict(X)
    return np.asarray([y_pred, 1 - y_pred])

In [8]:
def serving(inp):
    model = load()
    return predict(inp, model)

In [9]:
def no_pp(inp):
    return inp

image_up = gradio.inputs.ImageUpload(preprocessing_fn=no_pp)
label = gradio.outputs.Label(label_names=["benign", "malign"], num_top_classes=2)
iface = gradio.Interface(inputs=image_up, outputs=label, model_type="pyfunc", model=serving)

In [10]:
iface.launch(validate=False, inline=True, share=False)

NOTE: Gradio is in beta stage, please report all bugs to: gradio.app@gmail.com
Model is running locally at: http://localhost:7861/
To create a public link, set `share=True` in the argument to `launch()`.


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x1316866a0>,
 'http://localhost:7861/',
 None)

## Launch from config

In [11]:
import sys

def launch_from_config(config_path, inline=True, inbrowser=None, share=True):
#     sys.path.append(config_path)
    from gradio_config import load, predict, INPUT, OUTPUT
    model = load()
    
    iface = gradio.Interface(inputs=INPUT, outputs=OUTPUT, model_type="pyfunc", model=predict)
    
    iface.launch(inline=inline, inbrowser=inbrowser, share=share, validate=False)


In [12]:
launch_from_config("gradio_config.py")

NOTE: Gradio is in beta stage, please report all bugs to: gradio.app@gmail.com
Model is running locally at: http://localhost:7862/
Model available publicly at: https://22310.gradio.app (may take up to a minute for link to be usable)
